In [1]:
import numpy as np

# Радиус сферической поверхности Земли в километрах
earth_radius_km = 6378.1

# Координаты аэропортов в формате NumPy массивов
coordinates = np.array([
    (50.026, 8.543),        # FRA
    (39.862, -104.673),     # DEN
    (13.681, 100.747),      # BKK
    # Добавьте координаты для остальных аэропортов
])

# Функция для вычисления расстояния между двумя точками на поверхности Земли
def haversine(lat1, lon1, lat2, lon2):
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    distance_km = earth_radius_km * c
    return distance_km

# Функция для определения расстояния между аэропортами по их IATA кодам
def calculate_distance(iata1, iata2):
    i1 = np.where(iata_codes == iata1)[0][0]
    i2 = np.where(iata_codes == iata2)[0][0]
    
    lat1, lon1 = coordinates[i1]
    lat2, lon2 = coordinates[i2]
    
    distance_km = haversine(lat1, lon1, lat2, lon2)
    return distance_km

# Массив с кодами IATA аэропортов
iata_codes = np.array(["FRA", "DEN", "BKK", # Добавьте остальные коды
])

# Пример использования функции
iata1 = "FRA"
iata2 = "DEN"
distance = calculate_distance(iata1, iata2)
if distance is not None:
    print(f"Расстояние между аэропортами {iata1} и {iata2}: {distance} км")
else:
    print("Неверные коды аэропортов.")


Расстояние между аэропортами FRA и DEN: 8096.823321190586 км


In [3]:
import numpy as np
import pylab
import matplotlib.pyplot as plt

filename = 'wb-data.dat.txt'
data = np.genfromtxt(filename, skip_header=1, missing_values='..', delimiter=';')
years = np.linspace(1960, 2013, 54).astype(int)
cambodia_data, thailand_data, laos_data = data[:3], data[3:6], data[6:]

def plot_data(data, vaccine):
    pylab.plot(years, data[0,4:], lw=2, alpha=0.5, label='Cambodia')
    pylab.plot(years, data[1,4:], lw=2, alpha=0.5, label='Thailand')
    pylab.plot(years, data[2,4:], lw=2, alpha=0.5, label='Laos')
    pylab.xlabel('Year')
    pylab.ylabel(r'Immunization rate (\%)')
    pylab.title('Immunization rates in children: ({})'.format(vaccine))
    pylab.legend(loc=4)
    pylab.show()

plot_data(data[::3], 'BCG')
plot_data(data[1::3], 'Pol3')
plot_data(data[2::3], 'Measles')

AttributeError: module 'pylab' has no attribute 'plot'

In [1]:
import numpy as np
from datetime import datetime, timedelta
EPOCH = datetime(1970, 1, 1)

dt = np.dtype({'names': ['name', 'height', 'first_ascent',
                         'first_winter_ascent', 'latitude', 'longitude'],
               'formats': ['|S13', 'u2', 'f8', 'f8', 'f8', 'f8']
              })

def angle_str_to_float(s):
    """
    Convert an angle given in degrees, D, minutes, M, and seconds, S as a
    string 'DdMdSs' in to a float representing decimal degrees.
    """

    d, rem = s.split('d')
    m, rem = rem.split('m')
    s, rem = rem.split('s')
    return float(d) + float(m)/60 + float(s)/3600

def angle_float_to_str(abs_angle):
    """
    Convert a float representing an angle in degrees to a string of the form
    'DdMdSs' representing degrees, D, minutes, M, and seconds, S.

    """

    d = int(abs_angle)
    fmins = (abs_angle - d) * 60
    m = int(fmins)
    s = int(round((fmins - m) * 60))
    return '{D:d}d{M:d}m{S:d}s'.format(D=d, M=m, S=s)

def longlat_str_to_float(s):
    """
    Convert a longitude or latitude given as a string 'DdMmSsZ' (Z=N, E, S or W)
    into a float representing decimal degrees.

    """

    s = s.decode('utf-8')
    sgn = {'N': 1, 'E': 1, 'S': -1, 'W': -1}
    return sgn[s[-1]] * angle_str_to_float(s[:-1])


def latitude_float_to_str(angle):
    """
    Convert a latitude as a float between -90 and 90 degrees into a string
    giving degrees, minutes and seconds: 'DdMmSsZ' (Z=N or S).

    """

    hemisphere = 'N' if angle >=0 else 'S'
    return angle_float_to_str(abs(angle)) + hemisphere

def longitude_float_to_str(angle):
    """
    Convert a longitude as a float between -180 and 180 degrees into a string
    giving degrees, minutes and seconds: 'DdMmSsZ' (Z=E or W).

    """

    hemisphere = 'E' if angle >=0 else 'W'
    return angle_float_to_str(abs(angle)) + hemisphere

def date_str_to_float(s):
    """
    Converts a date represented as string formatted 'DD/MM/YYYY' to a number
    of seconds since "the epoch" (1/1/1970).
    """

    return (datetime.strptime(s.decode('utf-8'), '%d/%m/%Y') -
                            EPOCH).total_seconds()
def date_float_to_str(t):
    """
    Convert a time in seconds since the epoch to a string formatted
    'DD/MM/YYYY'

    """

    return datetime.strftime((EPOCH + timedelta(seconds=t)), '%d/%m/%Y')

peaks = np.genfromtxt('ex6-2-b-mountain-data.txt', dtype=dt, skip_header=12,
            converters={2: date_str_to_float,
                        3: date_str_to_float,
                        4: longlat_str_to_float,
                        5: longlat_str_to_float},
            delimiter=(13,5,14,12,13,11), skip_footer=1, autostrip=True)

ilowest = np.argmin(peaks['height'])
print('The lowest 8000 m peak is {:s} at {:4d} m'.format(
              peaks[ilowest]['name'].decode('utf-8'), peaks[ilowest]['height']))

peaks.sort(order='latitude')
print('The most Northerly peak is {:s} at {:s} deg North'.format(
            peaks[-1]['name'].decode('utf-8'),
            latitude_float_to_str(peaks[-1]['latitude'])))
print('The most Southerly peak is {:s} at {:s} deg North'.format(
            peaks[0]['name'].decode('utf-8'),
            latitude_float_to_str(peaks[0]['latitude'])))

peaks.sort(order='longitude')
print('The most Easterly peak is {:s} at {:s} deg East'.format(
            peaks[-1]['name'].decode('utf-8'),
            longitude_float_to_str(peaks[-1]['longitude'])))
print('The most Westerly peak is {:s} at {:s} deg East'.format(
            peaks[0]['name'].decode('utf-8'),
            longitude_float_to_str(peaks[0]['longitude'])))

peaks.sort(order='first_ascent')
print('Most recent first ascent: {:s} on {:s}'.format(
        peaks[-1]['name'].decode('utf-8'),
        date_float_to_str(peaks[-1]['first_ascent'])))

i = np.nanargmax(peaks['first_winter_ascent'])
print('Most recent first winter ascent: {:s} on {:s}'.format(
    peaks[i]['name'].decode('utf-8'),
    date_float_to_str(peaks[i]['first_winter_ascent'])))


dt2 = np.dtype({'names': ['name', 'height', 'first_ascent'],
                'formats': ['|S13', 'u2', '|S10']})
ordered_peaks = np.zeros(len(peaks), dtype=dt2)
ordered_peaks['name'] = peaks['name']
ordered_peaks['height'] = peaks['height'] * 3.2808399
ordered_peaks['first_ascent'] = [date_float_to_str(t) 
                                    for t in peaks['first_ascent']]
ordered_peaks = np.sort(ordered_peaks, order='height')[::-1]

print('+---------------+------------+--------------+')
print('| Peak          | height /ft | first ascent |')
print('+---------------+------------+--------------+')
for peak, height, first_ascent in ordered_peaks:
        print('| {:13s} | {:^10d} | {:^12s} |'.format(
            peak.decode('utf-8'), height, first_ascent.decode('utf-8')))
print('+---------------+------------+--------------+')

The lowest 8000 m peak is Shishapangma at 8027 m
The most Northerly peak is K2 at 35d52m57sN deg North
The most Southerly peak is Kangchenjunga at 27d42m9sN deg North
The most Easterly peak is Kangchenjunga at 88d8m54sE deg East
The most Westerly peak is Nanga Parbat at 74d35m21sE deg East
Most recent first ascent: Shishapangma on 02/05/1964
Most recent first winter ascent: Nanga Parbat on 16/02/2016
+---------------+------------+--------------+
| Peak          | height /ft | first ascent |
+---------------+------------+--------------+
| Everest       |   29028    |  29/05/1953  |
| K2            |   28251    |  31/07/1954  |
| Kangchenjunga |   28110    |  25/05/1955  |
| Lhotse        |   27939    |  18/05/1956  |
| Makalu        |   27837    |  15/05/1955  |
| Cho Oyu       |   26906    |  19/10/1954  |
| Dhaulagiri I  |   26794    |  13/05/1960  |
| Manaslu       |   26781    |  09/05/1956  |
| Nanga Parbat  |   26660    |  03/07/1953  |
| Gasherbrum I  |   26509    |  05/07/1958  